In [1]:
pip install tensorflow matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set image size for ResNet50
img_size = (224, 224)

# Define data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Define the directories for your dataset
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'

# Load images into the generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='binary'  # Since it's a binary classification (cup vs non-cup)
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='binary'
)


Found 4148 images belonging to 2 classes.
Found 780 images belonging to 2 classes.


In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# Load pre-trained ResNet50 model without the top layers (classifier)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model to avoid retraining the pre-trained layers
base_model.trainable = False

# Build the model by adding a classifier on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification: reusable or non_reusable
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 69s 1us/step


In [7]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/10


C:\Users\Shubham\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


129/129 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.5384 - loss: 0.8082 - val_accuracy: 0.5716 - val_loss: 0.6638
Epoch 2/10
  1/129 ━━━━━━━━━━━━━━━━━━━━ 2:59 1s/step - accuracy: 0.5312 - loss: 0.6937

C:\Users\Shubham\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5312 - loss: 0.6937 - val_accuracy: 0.8333 - val_loss: 0.4844
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 219s 2s/step - accuracy: 0.6303 - loss: 0.6307 - val_accuracy: 0.7826 - val_loss: 0.5078
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7188 - loss: 0.5595 - val_accuracy: 0.6667 - val_loss: 0.5477
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 222s 2s/step - accuracy: 0.6443 - loss: 0.6159 - val_accuracy: 0.7760 - val_loss: 0.5047
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7812 - loss: 0.5903 - val_accuracy: 0.5000 - val_loss: 0.7797
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.6777 - loss: 0.5879 - val_accuracy: 0.7799 - val_loss: 0.4858
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7500 - loss: 0.5414 - val_accuracy: 0.7500 - val_loss: 0.5119
Epoch 9/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 219s 2s/step - accuracy: 0.7094 - loss: 0.5627 - val_accuracy: 0.7826 - 

In [8]:
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {validation_accuracy * 100:.2f}%")

25/25 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.7701 - loss: 0.4889
Validation Accuracy: 77.69%


In [9]:
base_model.trainable = True
for layer in base_model.layers[:-10]:  # Unfreeze the last 10 layers
    layer.trainable = False

# Recompile the model after fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Continue training with the unfrozen layers
history_fine_tuned = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 255s 2s/step - accuracy: 0.7424 - loss: 0.5881 - val_accuracy: 0.5599 - val_loss: 1.1346
Epoch 2/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7812 - loss: 0.3695 - val_accuracy: 0.7500 - val_loss: 0.7014
Epoch 3/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.8375 - loss: 0.3762 - val_accuracy: 0.5547 - val_loss: 1.1927
Epoch 4/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9062 - loss: 0.2549 - val_accuracy: 0.6667 - val_loss: 0.9588
Epoch 5/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 247s 2s/step - accuracy: 0.8477 - loss: 0.3480 - val_accuracy: 0.7852 - val_loss: 0.4519


In [19]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load a new image
img_path = '6.jpg'
img = image.load_img(img_path, target_size=img_size)

# Convert the image to a numpy array and expand dimensions
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize

# Predict
prediction = model.predict(img_array)
if prediction < 0.5:
    print("Non-reusable cup")
else:
    print("Reusable cup")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
Reusable cup


In [20]:
model.save('reusable_cup_detector.h5')